## 기상요인에 따른 서울시 공공자전거 수요 분석 및 예측(DL)

### 라이브러리 호출

In [44]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout

### 데이터셋 확인

In [45]:
# 데이터프레임 리스트 초기화
df_list = []

In [47]:
# 데이터 로드
df_weather = pd.read_csv('dataset/OBS_ASOS_TIM_2022.csv', encoding='cp949')
df_jan = pd.read_csv('dataset/서울특별시 공공자전거 이용정보(시간대별)_2201.csv', encoding='cp949')
df_feb = pd.read_csv('dataset/서울특별시 공공자전거 이용정보(시간대별)_2202.csv', encoding='cp949')
df_mar = pd.read_csv('dataset/서울특별시 공공자전거 이용정보(시간대별)_2203.csv', encoding='cp949')
df_apr = pd.read_csv('dataset/서울특별시 공공자전거 이용정보(시간대별)_2204.csv', encoding='cp949')
df_may = pd.read_csv('dataset/서울특별시 공공자전거 이용정보(시간대별)_2205.csv', encoding='cp949')
df_jun = pd.read_csv('dataset/서울특별시 공공자전거 이용정보(시간대별)_2206.csv', encoding='cp949')
df_jul = pd.read_csv('dataset/서울특별시 공공자전거 이용정보(시간대별)_2207.csv', encoding='cp949')
df_aug = pd.read_csv('dataset/서울특별시 공공자전거 이용정보(시간대별)_2208.csv', encoding='cp949')
df_sep = pd.read_csv('dataset/서울특별시 공공자전거 이용정보(시간대별)_2209.csv', encoding='cp949')
df_oct = pd.read_csv('dataset/서울특별시 공공자전거 이용정보(시간대별)_2210.csv', encoding='cp949')
df_nov = pd.read_csv('dataset/서울특별시 공공자전거 이용정보(시간대별)_2211.csv', encoding='cp949')
df_dec = pd.read_csv('dataset/서울특별시 공공자전거 이용정보(시간대별)_2212.csv', encoding='cp949')

In [48]:
# 데이터프레임을 리스트에 추가
df_list.append(df_jan)
df_list.append(df_feb)
df_list.append(df_mar)
df_list.append(df_apr)
df_list.append(df_may)
df_list.append(df_jun)
df_list.append(df_jul)
df_list.append(df_aug)
df_list.append(df_sep)
df_list.append(df_oct)
df_list.append(df_nov)
df_list.append(df_dec)

In [49]:
for df in df_list:
  print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339969 entries, 0 to 1339968
Data columns (total 12 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   대여일자     1339969 non-null  object 
 1   대여시간     1339969 non-null  int64  
 2   대여소번호    1339969 non-null  int64  
 3   대여소명     1339969 non-null  object 
 4   대여구분코드   1339969 non-null  object 
 5   성별       842781 non-null   object 
 6   연령대코드    1339969 non-null  object 
 7   이용건수     1339969 non-null  int64  
 8   운동량      1339969 non-null  object 
 9   탄소량      1339969 non-null  object 
 10  이동거리(M)  1339969 non-null  float64
 11  이용시간(분)  1339969 non-null  int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 122.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1271376 entries, 0 to 1271375
Data columns (total 12 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   대여일자     1271376 non-null  object 
 1   대여시간     1271376 non

In [50]:
for df in df_list:
  print(df.head())

         대여일자  대여시간  대여소번호  ...   탄소량 이동거리(M) 이용시간(분)
0  2022-01-01     0   1725  ...  0.00     0.0      44
1  2022-01-01     0   1167  ...  0.51  2210.0      15
2  2022-01-01     0   1059  ...  0.25  1080.0       4
3  2022-01-01     0   3794  ...  0.50  2160.0      15
4  2022-01-01     0   4014  ...  0.22   961.3       7

[5 rows x 12 columns]
         대여일자  대여시간  대여소번호  ...   탄소량  이동거리(M) 이용시간(분)
0  2022-02-01     0    108  ...  0.38  1640.00      19
1  2022-02-01     0   1150  ...  0.03   111.20       7
2  2022-02-01     0   1530  ...  0.15   635.97       4
3  2022-02-01     0   2505  ...  0.20   866.95       5
4  2022-02-01     0    437  ...  0.61  2631.02      20

[5 rows x 12 columns]
         대여일자  대여시간  대여소번호  ...   탄소량  이동거리(M) 이용시간(분)
0  2022-03-01     0    846  ...  0.31  1315.63       9
1  2022-03-01     0   2000  ...  0.31  1330.00       7
2  2022-03-01     0   3671  ...  2.23  9617.17      98
3  2022-03-01     0    733  ...  0.23   970.85       6
4  2022-03-01     0   115

In [51]:
for df in df_list:
  print(df.tail())

               대여일자  대여시간  대여소번호  ...   탄소량  이동거리(M) 이용시간(분)
1339964  2022-01-31    23    436  ...  1.01  4350.00      28
1339965  2022-01-31    23    509  ...  0.93  4025.19      21
1339966  2022-01-31    23    274  ...  0.31  1350.00      12
1339967  2022-01-31    23   3623  ...  0.48  2080.00      25
1339968  2022-01-31    23   4527  ...  0.17   715.79       4

[5 rows x 12 columns]
               대여일자  대여시간  대여소번호  ...   탄소량  이동거리(M) 이용시간(분)
1271371  2022-02-28    23   1019  ...  0.29  1245.50       7
1271372  2022-02-28    23    226  ...  0.29  1254.67      14
1271373  2022-02-28    23   2145  ...  0.00     0.00       9
1271374  2022-02-28    23   1509  ...  0.00     0.00       4
1271375  2022-02-28    23    417  ...  0.18   770.00       6

[5 rows x 12 columns]
               대여일자  대여시간  대여소번호  ...   탄소량   이동거리(M) 이용시간(분)
2218631  2022-03-31    23   1616  ...  0.00      0.00      17
2218632  2022-03-31    23   2024  ...  4.15  17900.30      92
2218633  2022-03-31    23   2050  ..

### 데이터 필터링

In [52]:
# 결과를 저장할 데이터프레임 초기화
result_list = []

# 필요한 컬럼 선택 (대여일자, 대여시간, 이용건수)
for df in df_list:
  df_grouped = df.groupby(['대여일자','대여시간'],as_index=False)['이용건수'].sum()
  result_list.append(df_grouped)

In [53]:
result_list

[           대여일자  대여시간  이용건수
 0    2022-01-01     0   575
 1    2022-01-01     1   466
 2    2022-01-01     2   328
 3    2022-01-01     3   250
 4    2022-01-01     4   246
 ..          ...   ...   ...
 739  2022-01-31    19  1703
 740  2022-01-31    20  1505
 741  2022-01-31    21  1031
 742  2022-01-31    22   362
 743  2022-01-31    23   243
 
 [744 rows x 3 columns],
            대여일자  대여시간  이용건수
 0    2022-02-01     0   273
 1    2022-02-01     1   214
 2    2022-02-01     2   154
 3    2022-02-01     3    64
 4    2022-02-01     4    33
 ..          ...   ...   ...
 667  2022-02-28    19  4911
 668  2022-02-28    20  4037
 669  2022-02-28    21  3712
 670  2022-02-28    22  3330
 671  2022-02-28    23  1809
 
 [672 rows x 3 columns],
            대여일자  대여시간  이용건수
 0    2022-03-01     0  1322
 1    2022-03-01     1   885
 2    2022-03-01     2   328
 3    2022-03-01     3   147
 4    2022-03-01     4    78
 ..          ...   ...   ...
 739  2022-03-31    19  7697
 740  2022-03-31  